In [90]:
import numpy as np
import random
import gurobipy as gp
import time

In [91]:
def generate_instances(n, p, capacity_mean, capacity_stddev):
    # Generate positions for nodes
    nodes = np.random.rand(n, 2)

    # Generate capacities for clusters
    capacities = np.random.normal(capacity_mean, capacity_stddev, p)

    return nodes, capacities

In [92]:
def generate_weights(n, weight_mean, weight_stddev):
    # Generate weights for nodes
    weights = np.random.normal(weight_mean, weight_stddev, n)

    return weights

In [93]:
def distance(p, q):
    return np.sqrt(np.sum((p-q)**2))

def solve_ccp(nodes, capacities, weights, lambda_param):
    n = nodes.shape[0]
    p = len(capacities)

    # Create model
    model = gp.Model('ccp')

    # Create decision variables
    x = {}
    y = {}
    for i in range(n):
        for j in range(p):
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name=f'x[{i},{j}]')
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name=f'y[{i}]')

    # Set objective function
    obj = gp.quicksum(distance(nodes[i], nodes[j]) * x[i,j] for i in range(n) for j in range(p))
    obj += lambda_param * gp.quicksum(capacities[j]*y[j] for j in range(p))
    obj -= lambda_param * gp.quicksum(weights[i]*x[i,j] for i in range(n) for j in range(p))
    model.setObjective(obj, gp.GRB.MINIMIZE)

    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(p)) == 1, name=f'assign[{i}]')

    model.addConstr(gp.quicksum(y[j] for j in range(p)) <= p, name='num_clusters')

    for i in range(n):
        for j in range(p):
            model.addConstr(x[i,j] <= y[j], name=f'x_c[{i},{j}]')

    # Solve model
    model.optimize()
    # Extract solution
    clusters = []
    for j in range(p):
        cluster = [i for i in range(n) if x[i,j].X > 0.5]
        clusters.append(cluster)

    return clusters, model

In [94]:
def solve_with_heuristic(nodes, weights, capacities, p):
    n = len(nodes)

    # Initialize the clusters and the cluster weights
    clusters = [[nodes[i]] for i in range(n)]
    cluster_weights = [weights[i] for i in range(n)]

    # Sort the nodes by weight in decreasing order
    node_indices = np.argsort(weights)[::-1]

    # Iterate over the nodes
    for i in range(n):
        node = node_indices[i]

        # Find the cluster with the minimum weight
        min_cluster = np.argmin(cluster_weights)

        # If the weight of the cluster is less than p, merge the node with the cluster
        if cluster_weights[min_cluster] < p:
            clusters[min_cluster].append(node)
            cluster_weights[min_cluster] += weights[node]

    return clusters, cluster_weights

In [95]:
def evaluate_objective_function(model, nodes, clusters, cluster_weights):
    obj_value = 0
    for i in range(len(clusters)):
        cluster = clusters[i]
        weight = cluster_weights[i]
        for j in range(len(cluster)):
            for k in range(j+1, len(cluster)):
                node_j = cluster[j]
                node_k = cluster[k]
                obj_value += model(node_j, node_k, weight)
    return obj_value

In [96]:
# Study the stability and variability of the solutions
def study_stability(model, n, p, nodes, capacities, weights, num_trials):
    stability = []
    for _ in range(num_trials):
        # Generate random realizations of the weights
        random_weights = generate_weights(n, weights[0], weights[1])

        # Solve the model using the heuristic algorithm
        clusters, cluster_weights = solve_with_heuristic(nodes, random_weights, capacities, p)

        # Evaluate the performance of the solution using the objective function
        obj_value = evaluate_objective_function(model, nodes, clusters, cluster_weights)

        # Append the results to the stability list
        stability.append(obj_value)

    # Calculate the mean and standard deviation of the stability list
    mean = np.mean(stability)
    stddev = np.std(stability)

    return mean, stddev

In [97]:
def kmeans(data, k, max_iterations=100):
    # Initialize cluster centers
    cluster_centers = data[np.random.choice(data.shape[0], k, replace=False)]

    for iteration in range(max_iterations):
        # Calculate distances from each data point to cluster centers
        distances = np.array([np.linalg.norm(data - center, axis=1) for center in cluster_centers])

        # Assign each data point to the closest cluster center
        cluster_assignments = np.argmin(distances, axis=0)

        # Calculate new cluster centers
        new_cluster_centers = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])

        # Check for convergence
        if np.allclose(cluster_centers, new_cluster_centers):
            break

        cluster_centers = new_cluster_centers

    return cluster_assignments, cluster_centers

In [98]:
# Example usage
np.random.seed(42)

n = 200
p = 2
capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
weights = generate_weights(n, weight_mean, weight_stddev)

clusters, model = solve_ccp(nodes, capacities, weights, lambda_param)
print(model)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 601 rows, 600 columns and 1202 nonzeros
Model fingerprint: 0xe878e5bd
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-04, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 15.1698562
Presolve removed 601 rows and 600 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -12.3219 15.1699 
No other solutions better than -12.3219

Optimal solution found (tolerance 1.00e-04)
Best objective -1.232188968768e+01, best bound -1.232188968768e+01, gap 0.0000%
<gurobi.Model MIP instance ccp: 601 constrs, 600 vars, No pa

In [99]:
num_trials = 50 # Number of trials for stability study
mean, stddev = study_stability(model, n, p, nodes, capacities, (weight_mean, weight_stddev), num_trials)

print("Mean of the stability: ", mean)
print("Standard deviation of the stability: ", stddev)

TypeError: 'Model' object is not callable